Notebook for converting models to onnx  
(Obsolete as this is now also implemented in the main codebase)

In [1]:
from transformers.convert_graph_to_onnx import convert
from transformers import GPT2Tokenizer, GPT2LMHeadModel

from onnxruntime_tools import optimizer

from os import environ
from psutil import cpu_count
import torch
import torch.nn.functional as F
import numpy as np

from src.data_utils import encode, decode

environ["OMP_NUM_THREADS"] = str(cpu_count(logical=True))
environ["OMP_WAIT_POLICY"] = 'ACTIVE'
from onnxruntime import InferenceSession, SessionOptions, get_all_providers

In [2]:
model_path = "output/nl2bash/07-31_14:57:12"
model_name = "gpt2-medium"
model = GPT2LMHeadModel.from_pretrained(model_path)
model.to('cuda')
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

convert(framework="pt", model=model_path, tokenizer=model_name, output="onnx/gpt2.onnx", opset=11)

ONNX opset version set to: 11
Loading pipeline (model: output/nl2bash/07-31_14:57:12, tokenizer: gpt2-medium)
Using framework PyTorch: 1.5.1
Found input input_ids with shape: {0: 'batch', 1: 'sequence'}
Found input attention_mask with shape: {0: 'batch', 1: 'sequence'}
Found output output_0 with shape: {0: 'batch', 1: 'sequence'}
Found output output_1 with shape: {1: 'batch', 3: 'sequence'}
Found output output_2 with shape: {1: 'batch', 3: 'sequence'}
Found output output_3 with shape: {1: 'batch', 3: 'sequence'}
Found output output_4 with shape: {1: 'batch', 3: 'sequence'}
Found output output_5 with shape: {1: 'batch', 3: 'sequence'}
Found output output_6 with shape: {1: 'batch', 3: 'sequence'}
Found output output_7 with shape: {1: 'batch', 3: 'sequence'}
Found output output_8 with shape: {1: 'batch', 3: 'sequence'}
Found output output_9 with shape: {1: 'batch', 3: 'sequence'}
Found output output_10 with shape: {1: 'batch', 3: 'sequence'}
Found output output_11 with shape: {1: 'batch',

/opt/conda/lib/python3.7/site-packages/transformers/modeling_gpt2.py:149: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  w = w / (float(v.size(-1)) ** 0.5)
/opt/conda/lib/python3.7/site-packages/transformers/modeling_gpt2.py:151: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  mask = self.bias[:, :, ns - nd : ns, :ns]
/opt/conda/lib/python3.7/site-packages/torch/onnx/utils.py:912: UserWarning: Provided key attention_mask for dynamic axes is not a valid input/output name
  warnings.warn("Provided key {} for dynamic axes is not a valid input/output name".format

In [ ]:
optimized_model = optimizer.optimize_model("onnx/gpt2.onnx", model_type=model_name)
#optimized_model.convert_model_float32_to_float16()
optimized_model.save_model_to_file("onnx/opt.onnx")

In [3]:
def create_model_for_provider(model_path: str, provider: str) -> InferenceSession: 
    assert provider in get_all_providers(), f"provider {provider} not found, {get_all_providers()}"
    # Few properties than might have an impact on performances (provided by MS)
    options = SessionOptions()
    options.intra_op_num_threads = 1
    # Load the model as a graph and prepare the CPU backend 
    return InferenceSession(model_path, options, providers=[provider])

onnx_model = create_model_for_provider("onnx/gpt2.onnx", "CUDAExecutionProvider")

In [4]:
def onnx_predict(query):
    query = encode(query)
    model_inputs = tokenizer.encode_plus(query, return_tensors="pt")
    inputs_onnx = {k: v.cpu().detach().numpy() for k, v in model_inputs.items() if k=='input_ids'}

    # Run the model (None = get all the outputs)
    for _ in range(100):
        #print(inputs_onnx)
        logits = onnx_model.run(None, inputs_onnx)[0][:,-1,:]
        logits = model.lm_head(torch.tensor(logits).to('cuda'))
        log_probs = F.softmax(logits, dim=-1)
        _, prev = torch.topk(log_probs, k=1)
        token = prev.item()
        if token == 198:
            break
        inputs_onnx['input_ids'] = np.atleast_2d(np.append(inputs_onnx['input_ids'], prev.cpu().numpy()))
    
    return decode(tokenizer, inputs_onnx['input_ids'][0])

In [11]:
%%time
print(onnx_predict('Rename "file.txt" in directories "v_1", "v_2", and "v_3" each to "v_1.txt", "v_2.txt", and "v_3.txt" respectively and print the conversion'))

find v_1 v_2 v_3 -type d -exec bash -c 'mv "$1" "${1%.*}.txt"' -- {} \;
CPU times: user 635 ms, sys: 36.6 ms, total: 671 ms
Wall time: 669 ms


In [14]:
%load_ext line_profiler
%lprun -f onnx_predict onnx_predict('Rename "file.txt" in directories "v_1", "v_2", and "v_3" each to "v_1.txt", "v_2.txt", and "v_3.txt" respectively and print the conversion')

Timer unit: 1e-06 s

Total time: 0.695339 s
File: <ipython-input-8-67c3c8d5d989>
Function: onnx_predict at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def onnx_predict(query):
     2         1         13.0     13.0      0.0      query = encode(query)
     3         1       2150.0   2150.0      0.3      model_inputs = tokenizer.encode_plus(query, return_tensors="pt")
     4         1         32.0     32.0      0.0      inputs_onnx = {k: v.cpu().detach().numpy() for k, v in model_inputs.items() if k=='input_ids'}
     5                                           
     6                                               # Run the model (None = get all the outputs)
     7        37        229.0      6.2      0.0      for _ in range(100):
     8                                                   #print(inputs_onnx)
     9        37     656478.0  17742.6     94.4          logits = onnx_model.run(None, inputs_onnx)[0][:,-1,